In [1]:
import pandas as pd 
import os

In [2]:
# directory_conn = os.path.join('.', 'traffictracer', 'conn')
# direcrory_meta = os.path.join('.', 'traffictracer', 'Meta_statistics') 
# directory_wlan = os.path.join('.', 'traffictracer', 'WLAN_statistics') 
# direcrory_conn_in_out = os.path.join('.', 'traffictracer', 'conn_in_out') 
# direcrory_meta_ori = os.path.join('.', 'traffictracer', 'Meta') 

# begin_time = '24-10-30--09-52-28'

# conn_path = os.path.join(directory_conn, begin_time + '.csv')
# meta_path = os.path.join(direcrory_meta, 'Meta-' + begin_time + '.csv')
# wlan_path = os.path.join(directory_wlan, 'WLAN-' + begin_time + '.csv')
# conn_in_out_path = os.path.join(direcrory_conn_in_out, 'conn-in-out-' + begin_time + '.csv')
# meta_ori_path = os.path.join(direcrory_meta_ori, 'Meta-' + begin_time + '.csv') 

# df_conn = pd.read_csv(conn_path)
# df_meta = pd.read_csv(meta_path)
# df_wlan = pd.read_csv(wlan_path)
# df_meta_ori = pd.read_csv(meta_ori_path) 

# columns = ['Address A', 'Port A', 'Address B', 'Port B', 'Packets', 'Bytes', 'Stream ID', 'Rel Start', 'Duration', 'Flows'] 
# meta_columns = ['M ' + item for item in columns] 
# wlan_columns = ['W ' + item for item in columns] 

In [3]:
tshark_directory = os.path.join('.', 'traffictracer')
statistic_directory = ['WLAN_statistics', 'Meta_statistics'] 
ori_directory = ['WLAN', 'Meta'] 
port_directory = 'tshark_port' 
conn_directory = 'conn' 

columns = ['Address A', 'Port A', 'Address B', 'Port B', 'Packets', 'Bytes', 'Stream ID', 'Rel Start', 'Duration', 'Flows'] 
meta_columns = ['M ' + item for item in columns] 
wlan_columns = ['W ' + item for item in columns] 


In [4]:
def getSniInfo(df_ori: pd.DataFrame) -> dict: 
    """ Get Server Name Indication from original packet information. 
    """
    df_ori = df_ori.dropna(subset=['TCP Stream index']) 
    server_name_unique = df_ori['Server Name'].dropna().unique() 
    domain_stream_all = {key: [] for key in server_name_unique} 
    for name in domain_stream_all.keys(): # get the dictionary of sni: stream_ids
        stream_ids = df_ori.loc[df_ori['Server Name'] == name, 'TCP Stream index'] 
        domain_stream_all[name].extend(stream_ids) 
    stream_domain_all = {stream_id: domain for domain, stream_ids in domain_stream_all.items() for stream_id in stream_ids} # reverse the dictionary 
    return stream_domain_all

In [5]:
def addSni(df_meta_statistics: pd.DataFrame, dict_stream_domain: dict) -> set: 
    df_meta_statistics['Server Name'] = df_meta_statistics['M Stream ID'].map(dict_stream_domain) 
    return set(df_meta_statistics['Server Name']) 

In [6]:
def extractColumns(df: pd.DataFrame, df_meta, df_wlan) -> pd.DataFrame:     
    if all(col in df.columns for col in columns): # examine whether the columns have been inplaced 
        df_temp = df[columns].copy() 
        if df.equals(df_meta): 
            col_transfer = {k: v for k, v in zip(columns, meta_columns)} 
            df_temp.rename(columns=col_transfer, inplace=True) 
        elif df.equals(df_wlan): 
            col_transfer = {k: v for k, v in zip(columns, wlan_columns)} 
            df_temp.rename(columns=col_transfer, inplace=True) 
    return df_temp 

In [7]:
# df_meta_col = extractColumns(df_meta)
# df_wlan_col = extractColumns(df_wlan)

In [8]:
# stream_domain = getSniInfo(df_meta_ori) 
# sni_set = addSni(df_meta_col, stream_domain) 

In [9]:
def mergeData(df_conn: pd.DataFrame, df_meta: pd.DataFrame, df_wlan: pd.DataFrame) -> pd.DataFrame: 
    df_merge_1st = pd.merge(df_conn, df_meta, left_on='inRemotePort', right_on='M Port A', how='inner')
    df_merge_fin = pd.merge(df_merge_1st, df_wlan, left_on='outLocPort', right_on='W Port A', how='inner')
    return df_merge_1st, df_merge_fin 

In [10]:
# df_temp, df_conn_in_out_merge = mergeData(df_conn, df_meta_col, df_wlan_col)

In [11]:
# df_conn_in_out_merge.to_csv(conn_in_out_path, index=False)

In [13]:
for port_file in os.listdir(os.path.join(tshark_directory, port_directory)): 
    # Get timestamp (prefix) by port recording 
    if port_file.startswith('24-11-07'): 
        timestamp = port_file[:-4] 
        time_split = timestamp.rsplit('--', 1) 
        begin_time = time_split[0] 

        direcrory_conn_in_out = os.path.join('.', 'traffictracer', 'conn_in_out') 
        conn_in_out_path = os.path.join(direcrory_conn_in_out, 'conn-in-out-' + begin_time + '.csv')

        for sd in statistic_directory: 
            for statistic_file in os.listdir(os.path.join(tshark_directory, sd)): 
                if statistic_file.startswith('Meta-' + begin_time): 
                    df_meta = pd.read_csv(os.path.join(tshark_directory, sd, statistic_file)) 
                    print(f"Read {statistic_file}") 
                elif statistic_file.startswith('WLAN-' + begin_time): 
                    df_wlan = pd.read_csv(os.path.join(tshark_directory, sd, statistic_file)) 
                    print(f"Read {statistic_file}") 
                # else: 
                #     print("Read Nothing!")

        od = ori_directory[1] # Meta
        for ori_file in os.listdir(os.path.join(tshark_directory, od)): 
            if ori_file.startswith('Meta-' + begin_time) and ori_file.endswith('.csv'): 
                df_meta_ori = pd.read_csv(os.path.join(tshark_directory, od, ori_file)) 
                print(f"Read {ori_file}") 
        for conn_file in os.listdir(os.path.join(tshark_directory, conn_directory)): 
            if conn_file.startswith(begin_time): 
                df_conn = pd.read_csv(os.path.join(tshark_directory, conn_directory, conn_file)) 

        df_meta_col = extractColumns(df_meta, df_meta, df_wlan)
        df_wlan_col = extractColumns(df_wlan, df_meta, df_wlan) 
        stream_domain = getSniInfo(df_meta_ori) 
        sni_set = addSni(df_meta_col, stream_domain) 
        df_temp, df_conn_in_out_merge = mergeData(df_conn, df_meta_col, df_wlan_col) 
        df_conn_in_out_merge.to_csv(conn_in_out_path, index=False)

Read WLAN-24-11-07--10-19-16.csv
Read Meta-24-11-07--10-19-16.csv
Read Meta-24-11-07--10-19-16.csv
Read WLAN-24-11-07--10-25-29.csv
Read Meta-24-11-07--10-25-29.csv
Read Meta-24-11-07--10-25-29.csv
Read WLAN-24-11-07--10-32-20.csv
Read Meta-24-11-07--10-32-20.csv
Read Meta-24-11-07--10-32-20.csv
Read WLAN-24-11-07--11-08-34.csv
Read Meta-24-11-07--11-08-34.csv
Read Meta-24-11-07--11-08-34.csv
Read WLAN-24-11-07--11-24-21.csv
Read Meta-24-11-07--11-24-21.csv
Read Meta-24-11-07--11-24-21.csv
Read WLAN-24-11-07--11-31-59.csv
Read Meta-24-11-07--11-31-59.csv
Read Meta-24-11-07--11-31-59.csv
Read WLAN-24-11-07--15-15-47.csv
Read Meta-24-11-07--15-15-47.csv
Read Meta-24-11-07--15-15-47.csv
Read WLAN-24-11-07--15-36-37.csv
Read Meta-24-11-07--15-36-37.csv
Read Meta-24-11-07--15-36-37.csv
Read WLAN-24-11-07--15-38-38.csv
Read Meta-24-11-07--15-38-38.csv
Read Meta-24-11-07--15-38-38.csv
Read WLAN-24-11-07--15-41-25.csv
Read Meta-24-11-07--15-41-25.csv
Read Meta-24-11-07--15-41-25.csv
Read WLAN-

C:\Users\Raku\AppData\Local\Temp\ipykernel_56900\2212725561.py:25: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_ori = pd.read_csv(os.path.join(tshark_directory, od, ori_file))


Read Meta-24-11-07--15-46-14.csv
Read WLAN-24-11-07--15-52-05.csv
Read Meta-24-11-07--15-52-05.csv
Read Meta-24-11-07--15-52-05.csv
Read WLAN-24-11-07--15-58-43.csv
Read Meta-24-11-07--15-58-43.csv
Read Meta-24-11-07--15-58-43.csv
Read WLAN-24-11-07--16-04-47.csv
Read Meta-24-11-07--16-04-47.csv
Read Meta-24-11-07--16-04-47.csv
Read WLAN-24-11-07--16-55-07.csv
Read Meta-24-11-07--16-55-07.csv


C:\Users\Raku\AppData\Local\Temp\ipykernel_56900\2212725561.py:25: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meta_ori = pd.read_csv(os.path.join(tshark_directory, od, ori_file))


Read Meta-24-11-07--16-55-07.csv
Read WLAN-24-11-07--17-01-16.csv
Read Meta-24-11-07--17-01-16.csv
Read Meta-24-11-07--17-01-16.csv
Read WLAN-24-11-07--17-06-23.csv
Read Meta-24-11-07--17-06-23.csv
Read Meta-24-11-07--17-06-23.csv


为什么会出现重复的行？

df_conn中存在一些没走代理的流量，但是，这个流量inRemotePort相同，但是outRemotePort可能不同，但对于Wireshark针对meta截获的流量来说